In [99]:
#Import Libraries

import pandapower as pp
from pandapower import plotting as pplot
import pandas as pd
import numpy as np

In [206]:
#Basic Setup

model = "balearics_2" #balaeric_1 = existing network, balearics_2 = solar network
peak = 2 #select peak (1 = 13:20, 2 = 21:40)
PF = 0.9 #set Power Factor
sbv = 1.0 #Slack Bus nominal voltage
net = pp.create_empty_network() #Create empty network within pandapower

In [207]:
#Bus Setup

df_bus = pd.read_excel('{}.xls'.format(model),sheet_name='bus') #Import spreadsheet with bus details

buses = {} #Create dictionary to store buses
for i in range (0 , len(df_bus)): #Loop to create buses according to imported spreadsheet
    #key within buses dictionary set to bus designation from spreadsheet
    buses[df_bus.at[i,'bus']] = pp.create_bus(net , 
                vn_kv = df_bus.at[i , 'vn_kv'] , 
                name = df_bus.at[i , 'name'],
                geodata = (df_bus.at[i , 'lattitude'] , df_bus.at[i , 'longitude']))


In [208]:
#Slack Bus Assignment - Mainland Interconnector

pp.create_ext_grid(net, bus = buses['BMAC01'], vm_pu = sbv, name = 'Slack Bus Interconnector')
#Voltage set to 1.05pu to allow for volt drop across interconnector


0

In [209]:
#Load Assignment

df_load = pd.read_excel('{}.xls'.format(model),sheet_name='load') #Import spreadsheet with load details

for i in range (0 , len(df_load)): #Loop to assign loads to buses according to imported spreadsheet
    pp.create_load(net, bus = buses["{}".format(df_load.at[i , 'bus'])] , 
                    p_mw = df_load.at[i , 'P{}'.format(peak)] , #selects P value according to selected peak (P1 or P2)
                    q_mvar = df_load.at[i,'P{}'.format(peak)]*np.tan(np.arccos(PF)) , #calculates Q for given peak and power factor
                    name = df_load.at[i, 'name'])

#Reactor Assignment

df_reactor = pd.read_excel('{}.xls'.format(model),sheet_name='reactor') #Import spreadsheet with load details

for i in range (0 , len(df_reactor)): #Loop to assign reators to buses according to imported spreadsheet
    pp.create_load(net, bus = buses["{}".format(df_reactor.at[i , 'bus'])] , 
                    p_mw = 0 ,
                    q_mvar = df_reactor.at[i , 'Q{}'.format(peak)] , #selects Q value according to selected peak (Q1 or Q2)
                    name = df_reactor.at[i, 'name'])


In [210]:
#Generator Assignment - Existing Assets

#Existing Generation
df_gen = pd.read_excel('{}.xls'.format(model),sheet_name='gen') #Import spreadsheet with load details
for i in range (0 , len(df_gen)): #Loop to assign generators to buses according to imported spreadsheet
   # if df_gen.at[i, 'P{}'.format(peak)] != 0:
        pp.create_sgen(net, bus = buses["{}".format(df_gen.at[i , 'bus'])] , 
                        p_mw = df_gen.at[i , 'P{}'.format(peak)] , #selects P value according to selected peak (P1 or P2)
                        q_mvar = df_gen.at[i , 'Q{}'.format(peak)],
                        name = df_gen.at[i, 'name'])
    

    

In [211]:
#Generation Assignment - Solar and Batteries

#Proposed Solar
df_gen_solar = pd.read_excel('{}.xls'.format(model),sheet_name='gen_solar') #Import spreadsheet with load details
for i in range (0 , len(df_gen_solar)): #Loop to assign generators to buses according to imported spreadsheet
   # if df_gen.at[i, 'P{}'.format(peak)] != 0:
        pp.create_sgen(net, bus = buses["{}".format(df_gen_solar.at[i , 'bus'])] , 
                        p_mw = df_gen_solar.at[i , 'P{}'.format(peak)] , #selects P value according to selected peak (P1 or P2)
                        q_mvar = df_gen_solar.at[i , 'Q{}'.format(peak)],
                        name = df_gen_solar.at[i, 'name'])

#Proposed Batteries
df_gen_batt = pd.read_excel('{}.xls'.format(model),sheet_name='gen_battery') #Import spreadsheet with load details
for i in range (0 , len(df_gen_batt)): #Loop to assign generators to buses according to imported spreadsheet
   # if df_gen.at[i, 'P{}'.format(peak)] != 0:
        pp.create_sgen(net, bus = buses["{}".format(df_gen_batt.at[i , 'bus'])] , 
                        p_mw = df_gen_batt.at[i , 'P{}'.format(peak)] , #selects P value according to selected peak (P1 or P2)
                        q_mvar = df_gen_batt.at[i , 'Q{}'.format(peak)],
                        name = df_gen_batt.at[i, 'name'])

In [212]:
#Transformer Assignment

df_tran = pd.read_excel('{}.xls'.format(model),sheet_name='transformer') #Import spreadsheet with transformer details

for i in range (0 , len(df_tran)): #Loop to assign transformers to buses according to imported spreadsheet
    pp.create_transformer_from_parameters(net, 
                    hv_bus = buses["{}".format(df_tran.at[i , 'hv_bus'])] , 
                    lv_bus = buses["{}".format(df_tran.at[i , 'lv_bus'])] ,
                    sn_mva = df_tran.at[i , 'sn_mva'] , #apparent power rating - true power rating of interconnector with 0.8 PF
                    vn_hv_kv = df_tran.at[i , 'vn_hv_kv'] , #HV voltage
                    vn_lv_kv = df_tran.at[i , 'vn_lv_kv'] , #LV voltage
                    vkr_percent = df_tran.at[i , 'vkr%'] , #Real part of relative short circuit voltage
                    vk_percent = df_tran.at[i , 'vk%'] , #Relative short circuit voltage
                    pfe_kw = df_tran.at[i , 'pfe_kw'] , #Iron losses
                    i0_percent = df_tran.at[i , 'i0%'] , #Open loop losses (% of rated current)
                    name = df_tran.at[i, 'name'])


In [213]:
#Line Assignment

df_line = pd.read_excel('{}.xls'.format(model), sheet_name = 'line') #Import spreadsheet with line details

for i in range (0 , len(df_line)): #Loop to assign transformers to buses according to imported spreadsheet
    pp.create_line_from_parameters(net,
                    from_bus = buses["{}".format(df_line.at[i , 'from'])] , 
                    to_bus = buses["{}".format(df_line.at[i , 'to'])] , 
                    length_km = df_line.at[i , 'length_km'] ,
                    r_ohm_per_km = df_line.at[i , 'R/km'] ,
                    x_ohm_per_km = df_line.at[i , 'X/km'] ,
                    c_nf_per_km = df_line.at[i , 'C/km'] ,
                    max_i_ka = df_line.at[i , 'I_max'] ,
                    name = df_line.at[i , 'name'])

In [214]:
#n-1 Analysis

#Limits
vmax = 1.1
vmin = 0.95
max_ll = 100

critical = list()

lines = net.line.index

for l in lines:
    net.line.loc[l , "in_service"] = False
    pp.runpp(net , algorithm = 'gs')
    
    if net.res_bus.vm_pu.max() > vmax or net.res_bus.vm_pu.min() < vmin or net.res_line.loading_percent.max() > max_ll:
        critical.append(l)
    
    net.line.loc[l , "in_service"] = True

print(critical)    


[6, 7, 13]


In [215]:
#Run Powerflow algorithm (gauss-siedel)
test_line = 13

net.line.loc[test_line , "in_service"] = False
pp.runpp(net, algorithm='gs');
net.line.loc[test_line , "in_service"] = True

net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,-1.780954e+00,-8.469007e-01,1.782917e+00,0.742068,1.963654e-03,-0.104832,1.786864e-02,0.017467,0.017869,0.965440,-2.596899,0.967163,-2.509264,5.105324
1,1.780954e+00,8.469007e-01,-1.779755e+00,-0.908289,1.198981e-03,-0.061388,1.786864e-02,0.018125,0.018125,0.965440,-2.596899,0.964373,-2.647092,5.178520
2,3.850995e+00,1.749866e+00,-3.844167e+00,-1.815500,6.828206e-03,-0.065634,3.825846e-02,0.038563,0.038563,0.967163,-2.509264,0.964373,-2.647092,11.018068
3,-1.067860e+01,-4.935188e+00,1.078142e+01,4.991206,1.028178e-01,0.056018,1.064010e-01,0.105795,0.106401,0.967163,-2.509264,0.982366,-1.764340,30.400282
4,-1.200878e+01,-5.585646e+00,1.209517e+01,5.655063,8.638502e-02,0.069417,1.179368e-01,0.117515,0.117937,0.982366,-1.764340,0.993908,-1.209977,33.696235
5,4.623658e+00,-6.957034e-01,-4.610454e+00,-17.232945,1.320427e-02,-17.928648,4.115252e-02,0.158166,0.158166,0.993908,-1.209977,0.986627,-1.352575,21.967521
6,1.925174e+01,-7.630165e+01,-1.915061e+01,-82.146284,1.011314e-01,-158.447930,3.426888e-01,0.370269,0.370269,1.004387,-0.250207,0.996387,-1.094493,56.101354
7,1.925174e+01,-7.630165e+01,-1.915061e+01,-82.146284,1.011314e-01,-158.447930,3.426888e-01,0.370269,0.370269,1.004387,-0.250207,0.996387,-1.094493,56.101354
8,1.773521e+02,6.509061e+01,-1.768800e+02,-64.426071,4.720477e-01,0.664542,4.957849e-01,0.496394,0.496394,1.000000,0.000000,0.995222,-0.332869,46.391992
9,7.269182e+01,2.461536e+01,-7.247980e+01,-25.802931,2.120242e-01,-1.187570,2.023741e-01,0.203926,0.203926,0.995222,-0.332869,0.990085,-0.704660,19.058499


In [217]:
net.res_bus
#pplot.simple_plot(net)

,vm_pu,va_degree,p_mw,q_mvar
0,0.965440,-2.596899,-0.000000,-0.000000
1,0.967163,-2.509264,5.044686,2.443253
2,0.964373,-2.647092,5.623922,2.723790
3,0.982366,-1.764340,1.227365,0.594440
4,0.993908,-1.209977,19.043094,9.222992
5,0.996387,-1.094493,0.000000,150.000000
6,0.986627,-1.352575,4.610454,17.232945
7,1.000000,0.000000,-218.846386,82.332440
8,1.004387,-0.250207,0.000000,0.000000
9,0.964298,-3.021755,0.000000,0.000000


In [216]:
print("Bus Vm min = {}, max = {}, Average = {}".format(net.res_bus["vm_pu"].min(),net.res_bus["vm_pu"].max(),net.res_bus["vm_pu"].mean()))

Bus Vm min = 0.9487415942899902, max = 1.0043867891229599, Average = 0.9742940631719375


In [194]:
#Save Bus and Line data as csv's

#Comine bus and bus results dataframes to display names and inputs alongside outputs
df_bus = pd.concat([net.bus , net.res_bus], axis=1, sort=False)
df_bus.to_csv('results_bus_pk{}_pf{}.csv'.format(peak,PF))

#Comine line and line results dataframes to display names and inputs alongside outputs
df_line = pd.concat([net.line , net.res_line], axis=1, sort=False)
df_line.to_csv('results_line_pk{}_pf{}.csv'.format(peak,PF))

#Powerflow 
pp.to_excel(net, 'pp_res_pk{}_pf{}.xlsx'.format(peak,PF))

In [169]:
#Print key results

#maximum line usage i.e. closes to operating at capacity
line_max_pct = net.res_line["loading_percent"].max()
print("Maximum Line Load Percentage is {}% for {}".format(round(line_max_pct,1),df_line.loc[df_line['loading_percent'] == line_max_pct, 'name'].item()))

#maximum volt drop
max_vd = net.res_bus["vm_pu"].min()
print("Maximum volt drop is at {} receiving {}% of rated voltage".format(df_bus.loc[df_bus['vm_pu'] == max_vd, 'name'].item(), round(max_vd*100,1)))

ValueError: can only convert an array of size 1 to a Python scalar